### Notebook to get Jay's labels where he only made one correction, should be able to merge with Jaaba prediction to create good full ground truth labels

In [64]:
from pathlib import Path
from tqdm import tqdm
import shutil
import os
from scipy.io import loadmat
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
import pandas as pd

In [65]:
columns = ["mat_path", "hand_labels"]
df = pd.DataFrame(columns=columns)

In [66]:
df

,mat_path,hand_labels


In [67]:
mat_dir = Path("/data/caitlin/mat_files/")

In [68]:
def check_all_behaviors(ethogram: np.ndarray):
    count = 0
    for row in ethogram:
        if row.any():
            count += 1
    if count == 3:
        return True
    else:
        return False

In [69]:
mat_paths = sorted(mat_dir.glob('*'))
len(mat_paths)

696

In [70]:
mat_errors = list()
def get_hand_labels(mat_path): 
    hand_labels = dict()

    m = loadmat(mat_path)
    try:
        num_trials = m["data"]["trial"].shape[0]
        
    except KeyError:
        mat_errors.append(mat_path)
        return

    for i in tqdm(range(num_trials)):
        mat_trial_index = np.argwhere(m["data"]["trial"].ravel() == (i + 1))
        # Trial not found in JAABA data
        if mat_trial_index.size == 0:
            continue
        
        try:
            mat_trial_index = mat_trial_index.item()
        except ValueError:
            continue

        behaviors = sorted([b.split('_')[0] for b in m['data'].dtype.names if 'scores' in b])
        
        if len(behaviors) != 6:
            continue

        all_behaviors = [
            "Lift",
            "Handopen",
            "Grab",
            "Sup",
            "Atmouth",
            "Chew"
        ]

        sorted_behaviors = [b for b in all_behaviors if b in behaviors]

        ethograms = []

        for b in sorted_behaviors:
            behavior_index = m['data'].dtype.names.index(f'{b}_labl_label')
            row = m['data'][mat_trial_index][0][behavior_index]
            row[row == -1] = 0
            ethograms.append(row)

        sorted_behaviors = [b.lower() for b in sorted_behaviors]

        ethogram = np.hstack(ethograms).T

        if check_all_behaviors(ethogram):
            hand_labels[m["data"]["exp"][i][0][0]] = ethogram

    return hand_labels

In [71]:
mat_paths[0]

PosixPath('/data/caitlin/mat_files/M232_20170306.mat')

In [72]:
for mat in tqdm(mat_paths):
    hand_labels = get_hand_labels(str(mat))
    if hand_labels:
        df.loc[len(df.index)] = [mat.stem, hand_labels] 

100%|█████████████████████████████████████████████████████████████████████████████████████████| 85/85 [00:00<00:00, 5351.16it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5596.79it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5528.82it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5564.03it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5560.89it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5579.11it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5498.74it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████

In [73]:
mat_errors

['/data/caitlin/mat_files/M236_20170817.mat',
 '/data/caitlin/mat_files/M236_20170818.mat',
 '/data/caitlin/mat_files/M238_20170717.mat',
 '/data/caitlin/mat_files/M238_20170719.mat',
 '/data/caitlin/mat_files/M238_20170724.mat',
 '/data/caitlin/mat_files/M238_20170725.mat',
 '/data/caitlin/mat_files/M238_20170726.mat',
 '/data/caitlin/mat_files/M238_20170727.mat',
 '/data/caitlin/mat_files/M240_20170724FinalChecked.mat',
 '/data/caitlin/mat_files/M240_20170725FinalChecked.mat',
 '/data/caitlin/mat_files/M240_20170727FinalChecked.mat',
 '/data/caitlin/mat_files/M240_20170728FinalChecked.mat',
 '/data/caitlin/mat_files/M240_20170801FinalChecked.mat',
 '/data/caitlin/mat_files/M240_20170802FinalChecked.mat',
 '/data/caitlin/mat_files/M240_20170807FinalChecked.mat',
 '/data/caitlin/mat_files/M240_20170808FinalChecked.mat']

In [74]:
count = 0
for row in df.iterrows():
    count += len(row[1]["hand_labels"].keys())
count

724

In [75]:
df.to_hdf(path_or_buf="/data/caitlin/exactly3_hand_labels.hdf", key="df")

/tmp/ipykernel_3905448/1558508658.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['mat_path', 'hand_labels'], dtype='object')]

  df.to_hdf(path_or_buf="/data/caitlin/exactly3_hand_labels.hdf", key="df")


In [76]:
df = pd.read_hdf("/data/caitlin/exactly3_hand_labels.hdf")

In [77]:
mat_errors2 = list()
def get_hand_labels(mat_path): 
    hand_labels = dict()

    m = loadmat(mat_path)
    try:
        num_trials = m["data1"]["trial"].shape[0]
        
    except KeyError:
        mat_errors2.append(mat_path)
        return

    for i in tqdm(range(num_trials)):
        mat_trial_index = np.argwhere(m["data1"]["trial"].ravel() == (i + 1))
        # Trial not found in JAABA data
        if mat_trial_index.size == 0:
            continue
        
        try:
            mat_trial_index = mat_trial_index.item()
        except ValueError:
            continue

        behaviors = sorted([b.split('_')[0] for b in m['data1'].dtype.names if 'scores' in b])
        
        if len(behaviors) != 6:
            continue

        all_behaviors = [
            "Lift",
            "Handopen",
            "Grab",
            "Sup",
            "Atmouth",
            "Chew"
        ]

        sorted_behaviors = [b for b in all_behaviors if b in behaviors]

        ethograms = []

        for b in sorted_behaviors:
            behavior_index = m['data1'].dtype.names.index(f'{b}_labl_label')
            row = m['data1'][mat_trial_index][0][behavior_index]
            row[row == -1] = 0
            ethograms.append(row)

        sorted_behaviors = [b.lower() for b in sorted_behaviors]

        ethogram = np.hstack(ethograms).T

        if check_all_behaviors(ethogram):
            hand_labels[m["data1"]["exp"][i][0][0]] = ethogram

    return hand_labels

In [78]:
for mat in tqdm(mat_errors):
    hand_labels = get_hand_labels(Path(mat))
    if hand_labels:
        df.loc[len(df.index)] = [Path(mat).stem, hand_labels] 

100%|█████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5524.73it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████| 81/81 [00:00<00:00, 5571.04it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5588.59it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5629.47it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████| 86/86 [00:00<00:00, 5337.29it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5607.36it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5610.08it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████

In [79]:
mat_errors2

[]

In [80]:
df

,mat_path,hand_labels
0,M232_20170306,"{'M232_20170306_v029': [[0, 0, 0, 0, 0, 0, 0, ..."
1,M232_20170307,"{'M232_20170307_v022': [[0, 0, 0, 0, 0, 0, 0, ..."
2,M232_20170308,"{'M232_20170308_v020': [[0, 0, 0, 0, 0, 0, 0, ..."
3,M232_20170310,"{'M232_20170310_v055': [[0, 0, 0, 0, 0, 0, 0, ..."
4,M232_20170314,"{'M232_20170314_v011': [[0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
260,M238_20170717,"{'M238_20170717_v021': [[0, 0, 0, 0, 0, 0, 0, ..."
261,M238_20170725,"{'M238_20170725_v001': [[0, 0, 0, 0, 0, 0, 0, ..."
262,M238_20170726,"{'M238_20170725_v001': [[0, 0, 0, 0, 0, 0, 0, ..."
263,M240_20170725FinalChecked,"{'M240_20170725_v084': [[0, 0, 0, 0, 0, 0, 0, ..."


In [81]:
df.to_hdf(path_or_buf="/data/caitlin/exactly3_hand_labels.hdf", key="df")

/tmp/ipykernel_3905448/1558508658.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['mat_path', 'hand_labels'], dtype='object')]

  df.to_hdf(path_or_buf="/data/caitlin/exactly3_hand_labels.hdf", key="df")


In [82]:
df = pd.read_hdf("/data/caitlin/exactly3_hand_labels.hdf")

In [83]:
df

,mat_path,hand_labels
0,M232_20170306,"{'M232_20170306_v029': [[0, 0, 0, 0, 0, 0, 0, ..."
1,M232_20170307,"{'M232_20170307_v022': [[0, 0, 0, 0, 0, 0, 0, ..."
2,M232_20170308,"{'M232_20170308_v020': [[0, 0, 0, 0, 0, 0, 0, ..."
3,M232_20170310,"{'M232_20170310_v055': [[0, 0, 0, 0, 0, 0, 0, ..."
4,M232_20170314,"{'M232_20170314_v011': [[0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
260,M238_20170717,"{'M238_20170717_v021': [[0, 0, 0, 0, 0, 0, 0, ..."
261,M238_20170725,"{'M238_20170725_v001': [[0, 0, 0, 0, 0, 0, 0, ..."
262,M238_20170726,"{'M238_20170725_v001': [[0, 0, 0, 0, 0, 0, 0, ..."
263,M240_20170725FinalChecked,"{'M240_20170725_v084': [[0, 0, 0, 0, 0, 0, 0, ..."


In [84]:
count = 0
for row in df.iterrows():
    count += len(row[1]["hand_labels"].keys())
count

732